# Bag of Words in Julia

Julia is a very exciting new programming language that you should definitely check out, especially if you're a heavy Python user who wants more speed out of your code.  Julia's NLP ecosystem isn't as mature as Python's or R's yet, so expect some rough edges, and expect to re-implement a lot of stuff yourself.  Julia is designed less around large, monolithic libaries (like Python's `scikit-learn`), and more around small, _composable_ libraries that you combine piece-by-piece.  There's also a stronger culture of not re-inventing the wheel; if something can be implemented in a few lines of code by the end user, then it's more likely to be left out of the library.

All this means: we'll need to do things like implementing a string-to-sparse matrix conversion ourself.  Fortunately this is pretty easy to do, as we'll see.

Note: I'll be using a lot of semicolons after the last line of each cell.  In Julia, assignment operations return the assigned value, and Jupyter wants to print that out.  A trailing semicolon after a line will suppress this otuput.  I'm also going to use type annotations pretty heavily--unnecessarily heavily, for anyone who's already a Julia programmer--just to make things more explicit.

In [1]:
# Julia uses the Pkg module of its standard library to do project-specific
# package dependencies.  This is like `renv` in R, and kind of like the various
# (non-conda) virtual environments in Python.
using Pkg
Pkg.activate(".")

  Activating project at `C:\Users\andersonh\Documents\UA Projects\LAK 2023\demos\julia`


In [2]:
# Run this cell if you need to install the dependencies, otherwise skip it
# Pkg.add("CSV")            # parse CSV files
# Pkg.add("DataFrames")     # dataframes
# Pkg.add("MLJ")            # general machine learning framework
# Pkg.add("MLJScikitLearnInterface") # interface to scikit-learn, so we can use sklearn models
# Pkg.add("Pipe")           # macros for better piping syntax
# Pkg.add("ProgressMeter")  # progress bars
# Pkg.add("Snowball")       # interface to the Snowball stemming libary
# Pkg.add("WordTokenizers") # some common tokenization algorithms

In [3]:
# load the data.  This step will take a while the first time 
# you run it, while Julia pre-compiles things.
using CSV
using DataFrames

train = DataFrame(CSV.File("../../data/train.csv"))
test = DataFrame(CSV.File("../../data/test.csv"))
val = DataFrame(CSV.File("../../data/validation.csv"));

In [4]:
println(train[1:10, :])

10×8 DataFrame
 Row │ review_id   product_id          reviewer_id          stars  review_body                        review_title                       language  product_category    
     │ String15    String31            String31             Int64  String                             String                             String3   String31            
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ en_0964290  product_en_0740675  reviewer_en_0342986      1  Arrived broken. Manufacturer def…  I'll spend twice the amount of t…  en        furniture
   2 │ en_0690095  product_en_0440378  reviewer_en_0133349      1  the cabinet dot were all detache…  Not use able                       en        home_improvement
   3 │ en_0311558  product_en_0399702  reviewer_en_0152034      1  I received my first order of thi…  The product is junk.               en        home
   4 │ e

In [5]:
using Pipe
using ProgressMeter
using Snowball

const STOPWORDS = map(
    x -> stem(Stemmer("english"), x),
    split("""i me my myself we our ours ourselves you your yours yourself yourselves he him his himself she her hers herself it its itself they them their theirs themselves what which who whom this that these those am is are was were be been being have has had having do does did doing a an the and but if or because as until while of at by for with about against between into through during before after above below to from up down in out on off over under again further then once here there when where why how all any both each few more most other some such no nor not only own same so than too very s t can will just don should now aren isn weren""")
)

# A simple preprocessing function that generally looks like Gensim's default
# preprocessing steps.
preprocess(s::String, stemmer::Stemmer) :: Vector{String} = @pipe (
    s
    |> lowercase(_)
    |> replace(_, r"[^a-z]+" => " ")
    |> split(_)
    |> map(x -> stem(stemmer, x), _)
    |> filter(x -> length(x) >= 3, _)
    |> filter(x -> !(x in STOPWORDS), _)
)

train_tokens = @showprogress "Preprocessing training data" [
     preprocess(i, Stemmer("english")) for i in train[!, :review_body]
]
test_tokens = @showprogress "Preprocessing testing data" [
     preprocess(i, Stemmer("english")) for i in test[!, :review_body]
]

train_tokens[1]

Preprocessing training data 100%|████████████████████████| Time: 0:00:10:39m
Preprocessing testing data 100%|█████████████████████████| Time: 0:00:00


54-element Vector{String}:
 "arriv"
 "broken"
 "manufactur"
 "defect"
 "two"
 "leg"
 "base"
 "complet"
 "form"
 "way"
 "insert"
 "caster"
 "unpackag"
 ⋮
 "miss"
 "though"
 "hesit"
 "buy"
 "make"
 "wonder"
 "miss"
 "structur"
 "support"
 "imped"
 "assembl"
 "process"

We can re-implement a document frequency filter pretty compactly.  This will replicate the general functionality of `gensim.Dictionary` and `gensim.Dictionary.filter_extremes()` from Python.

In [6]:
# function to count how many times each token appears across
# the corpus.  `iterable` is e.g. an array of array of strings.
function counter(iterable)
    counts = Dict()
    for i ∈ iterable
        counts[i] = get(counts, i, 0) + 1
    end
    return counts
end

function remove_extreme!(counts::Dict, threshold::Number, comparison::Function)
    if threshold < 1
        threshold = sum(values(counts)) * threshold
    end
    
    for (k, v) ∈ pairs(counts)
        if comparison(v, threshold)
            delete!(counts, k)
        end
    end
end

remove_frequent!(counts, thresh) = remove_extreme!(counts, thresh, >)
remove_rare!(counts, thresh) = remove_extreme!(counts, thresh, <)

# to get document frequencies, convert each document to a Set()--which
# deduplicates entries--and then run the Counter.
word_counts = counter(tok for doc ∈ train_tokens for tok ∈ Set(doc))
remove_frequent!(word_counts, 0.5)
remove_rare!(word_counts, 10)

In [7]:
# bag of words matrix time!  Need to convert word into row indices
# and documents into columns (Julia arrays are column-first, unlike
# Python/R which are row-first).
using SparseArrays

# Use a dict of token => index mappings to convert a document into a
# vector of row indices.  This will drop any tokens not in the `ocab`
# dict.
function doc2bow(vocab::Dict{String, Int}, doc::Vector{String}) :: Dict{Int, Int}
    return Dict(
        vocab[tok] => count
        for (tok, count) in counter(doc)
        if tok ∈ keys(vocab)
    )
end

function tokens2bow(vocab::Dict{String, Int}, docs::Vector{Vector{String}}) :: SparseMatrixCSC
    # convert each document into a dict of token_index => count pairs
    bow = [doc2bow(vocab, doc) for doc ∈ docs]

    # set up the "internal" arrays for the sparse matrix.
    # This is a pretty standard sparse matrix format, but you
    # may need to read some documentation for it to make sense
    # if you haven't seen it before.
    colptr = zeros(Int, length(bow) + 1)
    rowval = zeros(Int, sum(length.(bow)))
    nzval  = zeros(UInt16, sum(length.(bow)))
    colptr[1] = 1
    
     # indices that we'll advance through as we update the above arrays
    rowval_ptr = 1
    colptr_ptr = 2
    
    # update the colptr/rowval arrays
    for doc in bow
        for (row_idx, val) in doc
            rowval[rowval_ptr] = row_idx
            nzval[rowval_ptr] = val
            rowval_ptr += 1
            colptr[colptr_ptr] += 1
            # println("[$row_idx, $(colptr_ptr-1)]=$val")
        end
        colptr_ptr += 1
    end
    
    return SparseMatrixCSC(
        length(vocab),
        length(bow),
        cumsum(colptr),
        rowval,
        nzval,
    )
end

# token-to-index mapping
vocab = Dict(j => i for (i, j) in enumerate(keys(word_counts)))

train_bow = tokens2bow(vocab, train_tokens)
test_bow = tokens2bow(vocab, test_tokens)

println(size(train_bow))
println(size(test_bow))

(7704, 200000)
(7704, 5000)


We could use `ScikitLearn.jl`, which provides a direct interface to scikit-learn models.  But as of writing this (early 2023), ScikitLearn.jl still has pretty bad support for sparse matrices.  It'll tend to convert them to dense matrices, which is bad--it'll cause our memory usage to explode.

So instead, we'll use a Decision Tree from the DecisionTree.jl package (via the MLJ.jl interface, which wraps DecisionTree.jl and a lot of other libraries in a nicer interface), which will support sparse feature arrays.  But since this takes a long time to run--decision trees are generally slow when there are very large numbers of features--we'll also throw in some Singular Value Decomposition to reduce our number of features down to a more reasonable 100.

In [8]:
using MLJ

model = @load DecisionTreeClassifier pkg=DecisionTree
svd = @load TSVDTransformer pkg=TSVD
model = machine(
    Pipeline(svd(nvals=300), model()),
    coerce(transpose(train_bow), Continuous),
    coerce(train[!, :stars], Multiclass),
)

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\andersonh\.julia\packages\MLJModels\8Nrhi\src\loading.jl:159


import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\andersonh\.julia\packages\MLJModels\8Nrhi\src\loading.jl:159


import MLJTSVDInterface ✔


untrained Machine; does not cache data
  model: ProbabilisticPipeline(tsvd_transformer = TSVDTransformer(nvals = 300, …), …)
  args: 
    1:	Source @896 ⏎ AbstractMatrix{Continuous}
    2:	Source @005 ⏎ AbstractVector{Multiclass{5}}


In [ ]:
fit!(model)

┌ Info: Training machine(ProbabilisticPipeline(tsvd_transformer = TSVDTransformer(nvals = 300, …), …), …).
└ @ MLJBase C:\Users\andersonh\.julia\packages\MLJBase\uxwHr\src\machines.jl:492
┌ Info: Training machine(:tsvd_transformer, …).
└ @ MLJBase C:\Users\andersonh\.julia\packages\MLJBase\uxwHr\src\machines.jl:492


In [ ]:
preds = predict(model, coerce(transpose(test_bow), Continuous));
println(preds[1])

The predictions are a `UnivariateFinite` type--which is actually a kind of distribution.  This intuitively makes a lot of sense, ince a (pseudo-)probabilistic prediction is basically just a distribution over possible classes.  To convert these to a hard-margin classification we can just call the `mode` function, which extract the value with the highest probability mass from the distribution.  This returns a `CategoricalValue`--which is part of the `CategoricalArrays` library that handles categorical data, but we can just directly compare to our various integer class labels and not worry about this.  Unless we want to treat the predictions as numbers, e.g. for calculating an $r^2$ or Mean Absolute Error score.  In those cases we can just use the `unwrap()` function from `CategoricalValues`.

In [ ]:
using CategoricalArrays

preds = mode.(preds)
println("""
    Accuracy: $(mean(preds .== test[!, :stars]))
    Macro F1: $(macro_f1score(preds, test[!, :stars]))
    R^2:      $(rsq(unwrap.(preds), test[!, :stars]))
    MAE:      $(mean(abs.(unwrap.(preds) .- test[!, :stars])))
""")

This model did quite a lot worse than the `BernoulliNB` from `scikit-learn`, but then again, we did have to do some extra steps due to still-poor Sparse Array support in Julia's ML libraries.  But there are some very cool machine learning libaries we could have used instead, like `Flux.jl`, which lets us write very Pytorch-like neural networks, or `SimpleChains.jl` for extremely fast implementations of simpler neural network architectures.  (the kind you'd get from only using, e.g., `torch.Sequential` or `keras.Sequential` in Python).